# **Classifier Small Dataset** 

In [1]:
import os
import sys

sys.path.insert(0, "..\\Scripts")

from torchvision import transforms
from torch.utils.data import DataLoader
from ImageDatastore import ImageDatastore

## **Dataset**

In [2]:
batch_size = 512

In [3]:
# transform = transforms.Compose(
#     [
#         transforms.Resize(
#             (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
#         ),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ]
# )
transform = transforms.Compose(
    [
        transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.Pad(padding=(0, 0, 0, 0), fill=0),
        transforms.CenterCrop((232, 232)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [ ]:
train_small = ImageDatastore('train', transform=transform)
train_small_loader = DataLoader(train_small, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)

In [11]:
val_small = ImageDatastore('val_set', transform=transform)
val_small_loader = DataLoader(val_small, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [12]:
val_deg = ImageDatastore('val_degradate', transform=transform)
val_deg_loader = DataLoader(val_deg, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

## **Feature extraction**

### **Hand Crafted feature extraction**

In [7]:
from BaseFeatureExtractor import BaseFeatureExtractor

In [ ]:
feature_extractor = BaseFeatureExtractor("sift", 16, (232, 232), 50)

In [10]:
paths, labels = {"train_small": [], "val_small": [], "val_deg": []}, {
    "train_small": [],
    "val_small": [],
    "val_deg": [],
}

for i, row in train_small.labels.iterrows():
    paths["train_small"].append(
        os.path.join("..", "Dataset", "train_set", row["Image"])
    )
    labels["train_small"].append(row["Label"])

for i, row in val_small.labels.iterrows():
    paths["val_small"].append(os.path.join("..", "Dataset", "val_set", row["Image"]))
    labels["val_small"].append(row["Label"])

for i, row in val_deg.labels.iterrows():
    paths["val_deg"].append(
        os.path.join("..", "Dataset", "val_set_degraded", row["Image"])
    )
    labels["val_deg"].append(row["Label"])

In [ ]:
x_train, y_train, centroids = feature_extractor.compute_features(
    paths["train_small"], labels["train_small"]
)
# x_train, y_train = feature_extractor.compute_feature_color_lbp(train_small)

x_train.shape, y_train.shape

100%|██████████| 5020/5020 [00:25<00:00, 197.51it/s]


In [ ]:
x_test, y_test = feature_extractor.compute_feature_sift_or_surf_test(
    paths["val_small"], labels["val_small"], centroids
)
# x_test, y_test = feature_extractor.compute_feature_color_lbp(val_small)
x_test.shape, y_test.shape

100%|██████████| 11994/11994 [06:23<00:00, 31.30it/s]


((11994, 16), (11994,))

In [ ]:
x_deg, y_deg = feature_extractor.compute_feature_sift_or_surf_test(
    paths["val_deg"], labels["val_deg"], centroids
)
# x_deg, y_deg = feature_extractor.compute_feature_color_lbp(val_deg)
x_deg.shape, y_deg.shape

100%|██████████| 11994/11994 [07:54<00:00, 25.28it/s]


((11994, 16), (11994,))

### **Neural Feature Extraction**

In [7]:
from NeuralFeatureExtractor import MobileNetFeatureExtractor, AlexNetFeatureExtractor, ResNetFeatureExtractor

In [8]:
feature_extractor = MobileNetFeatureExtractor()
# feature_extractor = AlexNetFeatureExtractor()
# feature_extractor = ResNetFeatureExtractor()

In [9]:
x_train, y_train = feature_extractor.compute_features(train_small_loader)
x_train.shape, y_train.shape

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [03:05<00:00, 18.57s/it]


((5020, 1280), (5020,))

In [13]:
x_test, y_test = feature_extractor.compute_features(val_small_loader)
x_test.shape, y_test.shape

100%|██████████| 24/24 [05:58<00:00, 14.94s/it]


((11994, 1280), (11994,))

In [ ]:
x_deg, y_deg = feature_extractor.compute_features(val_deg_loader)
x_deg.shape, y_deg.shape

100%|██████████| 24/24 [05:14<00:00,  8.68s/it]

In [ ]:
import numpy as np
np.save("x_train_small.npy", x_train)
np.save("y_train_small.npy", y_train)
np.save("x_val.npy", x_test)
np.save("y_val.npy", y_test)
np.save("x_val_deg.npy", x_deg)
np.save("y_val_deg.npy", y_deg)

## **Models**

### **k-NN Classifier**

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score

In [13]:
knn = KNeighborsClassifier(n_neighbors=51, n_jobs=-1, weights='distance', metric='cosine')

In [14]:
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=51,
                     weights='distance')

In [15]:
knn_pred = knn.predict(x_test)
print(accuracy_score(y_test, knn_pred))

0.27447056861764213


In [16]:
knn_score = knn.predict_proba(x_test)
print(top_k_accuracy_score(y_test, knn_score, k=5))

0.532349508087377


In [18]:
knn_pred_deg = knn.predict(x_deg)
print(accuracy_score(y_deg, knn_pred_deg))

0.20018342504585626


In [19]:
knn_score_deg = knn.predict_proba(x_deg)
print(top_k_accuracy_score(y_deg, knn_score_deg, k=5))

0.4024512256128064


### **Tree Classifier**

In [20]:
from sklearn.ensemble import RandomForestClassifier

tree = RandomForestClassifier(criterion='gini', n_estimators=100, n_jobs=-1)

In [21]:
tree.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [22]:
tree_pred = tree.predict(x_test)
print(accuracy_score(y_test, tree_pred))

0.16649991662497915


In [23]:
tree_score = tree.predict_proba(x_test)
print(top_k_accuracy_score(y_test, tree_score, k=5))

0.3416708354177089


In [24]:
tree_pred_deg = tree.predict(x_deg)
print(accuracy_score(y_deg, tree_pred_deg))

0.11589127897281974


In [25]:
tree_score_deg = tree.predict_proba(x_deg)
print(top_k_accuracy_score(y_deg, tree_score_deg, k=5))

0.23870268467567116


### **Ensemble Classifier**